## Описаение 
Данный блок производт парсинг обучающего набора данных для проекта [Проект 7. Выбираем авто выгодно](https://lms.skillfactory.ru/courses/course-v1:Skillfactory+DST-12+11MAR2020/courseware/c16441cf4f0a4f8486955f2be47f1cf0/67e09c15d9cd4b8eb691417898f4dfc2/1?activate_block_id=block-v1%3ASkillfactory%2BDST-12%2B11MAR2020%2Btype%40vertical%2Bblock%40f820a06d37d84dd98ab6acaa03392b5e)

Данные сбираются на сайте https://auto.ru/,  
из них формируется *pd.DataFrame* со следуюшими колонками:   
  
**bodytype** -   наименование типа кузова  
**brand** -  наименование марки  
**car_url** -  ссылка на обьявления о продаже  
**color** -  ссылки на обьявления о продаже  
**engineDisplacement** -  обьем двигателя  
**enginePower** -  мощность двигателя  
**equipment_dict** -  словарь с перечислением оснащения автомобиля.  
**fuelType** -  тип топлива  
**mileage** -  пробег авто  
**modelDate** -  год начала выпуска модели  
**model_name** -  наименование модели  
**numberOfDoors** -  количество дверей  
**productionDate** -  Год производства автомобиля  
**sell_id** -  содержит id обьявления   
**vehicleTransmission** -  содержит id обьявления  
**vendor** -  обобщающий признак: принадлежность марки к европейским либо японским маркам  
**Владельцы** -  количество владельцев авто  
**ПТС** -  Колонка содержит ('Оригинал', 'Дубликат') категорию ПТС  
**Привод** -   категория привода  
**Руль** -   категрию право- или левосторонности управления  
**offerprice** - цена продпажи (целевая переменная)

# Импорт библиотек, установка параметров, определение функций
## Импорт библиотек

In [2]:
import pandas as pd
import json
import numpy as np
import time
import re

from selenium import webdriver
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

## Функции

In [10]:
def get_marks_models():
    '''
    возврашщает pd.Series в котором 
    индекс - название марки автомобиоя, 
    значения - списки названий моделей для каждой маркию
    
    '''
    url_for_marks_models = 'https://auto.ru/catalog/cars/all/'
    marks_models = dict()
    
    option = webdriver.ChromeOptions()
    chrome_prefs = {}
    option.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
#     option.add_argument('headless')

    driver = webdriver.Chrome(executable_path=executable_path)  # открываем driver
#     driver.maximize_window()                    # масксимизируем окно

    for pages_num in range(1,20):
        
        if pages_num==1: url = url_for_marks_models + '?view_type=list'
        else:            url = url_for_marks_models + '?page_num='+ str(pages_num) +'&view_type=list'
        
        driver.get(url)                         # открываем страницу по url
        res = driver.execute_script("return document.body.innerHTML;") # получаем html
        soup = BeautifulSoup(res, 'html.parser') # создаем обьект bs4.BeautifulSoup
        
        # Список html-ек перечня моделей каждой марки на странице:
        marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc') 
    
        if not marks_on_page_list: 
            break
        
        for mark_html in marks_on_page_list:
            models_of_mark = mark_html.find_all('a', class_='link_theme_auto') # список html-ек моделей марки
        
            link_for_mark_name = models_of_mark[0].get('href')  # линк первой модели, содержит обозначене марки на auto.ru
            mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
            mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
            mark_name = link_for_mark_name[mark_start:mark_end].upper() # получение обозначения марки  
        
            models_list = []
            for model in models_of_mark:
                link_theme_auto = model.get('href')                 # линк модели, содержит обозначение модели на auto.ru
                model_start = mark_end+1                            # первый символ обозначения модели
                model_end = link_theme_auto.find('/', model_start)  # последний символ обозначения модели
                model_name = link_theme_auto[model_start:model_end].upper() # получение обозначения модели
                models_list.append(model_name) 
            marks_models[mark_name] = models_list
        time.sleep(1) 
    # Закрываем процесс браузера:
    driver.quit()
    return marks_models

def get_generation_year(model_url,driver):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название модели с указанием поколения, 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
        
    model_url - ссылка на страницу со списокм поколений модели
    '''
    driver.get(model_url+'?output_type=models_list') # открываем страницу по url
    models_list_res = driver.execute_script("return document.body.innerHTML;") # получаем html        
    # создаем обьект bs4.BeautifulSoup из html: 
    models_list_bs = BeautifulSoup(models_list_res, 'html.parser') 
    # получаем список html-ек описаний для всех поколений модели  
    posting_tag = models_list_bs.find_all('h3',class_ = "ListingItemGroup__title")
    # формируем списки с указанием поколения модели, типом кузова поколения и годом начала выпуска поколения
    # (необходимость указания типа кузова вызвана тем, что разные кузова зачастую переходят на следующее 
    #  поколение в разные годы)
    data = [
    [tag.find('a',class_ = 'ListingItemTitle__link').text,             # поколение
     tag.find('div',class_ = "ListingItemGroup__subtitle").text[:      # кузов
        tag.find('div',class_ = "ListingItemGroup__subtitle").text.find(' •')],
     int(tag.find('div',class_ = "ListingItemGroup__subtitle").text    # год
        [tag.find('div',class_ = "ListingItemGroup__subtitle").text.find('(')+1:
        tag.find('div',class_ = "ListingItemGroup__subtitle").text.find('(')+5])] 
        for tag in posting_tag]
    result = pd.DataFrame(data,columns = ['full_name','bodytype','generation_year'])
    return result 

def get_model_generation_year(marks_models_for_parsing):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название марки и модели с указанием поколения 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
    
    marks_models_for_parsing - pd.Series в котором
        индекс: марка автомобиля
        значение: список моделей данной марки
    '''
    driver = webdriver.Chrome(executable_path=executable_path)  # запускаем процесс браузера
    driver.maximize_window()                    # масксимизируем окно    
    
    model_generation_year = pd.DataFrame(columns = ['full_name','bodytype','generation_year'])
    
    for mark in marks_models_for_parsing:
        print(mark, end=' | ')
        
        for model in marks_models_for_parsing[mark]:
            model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'
            tmp = get_generation_year(model_url,driver)
            model_generation_year = model_generation_year.append(tmp)
            time.sleep(1)
            
    driver.quit()    # закрываем процесс браузера
    
    return model_generation_year

def get_features_from_ticket(ticket_url, driver):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
    service : обьект selenium.webdriver.chrome.service
        
    '''
    features = pd.Series(index = data_columns)
    # получем html карточки текщего обьявления
    driver.get(ticket_url)
    # Находим и кликаем 'Все опции'
    try:
        butt = driver.find_element(By.CLASS_NAME, 'ComplectationGroupsDesktop__cut') 
        butt.click() 
    except Exception: pass
    # Получаем содержимое html-страницы:
    ticket_res = driver.execute_script("return document.body.innerHTML;")
    # Закрываем процесс браузера:
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res, 'html.parser')  # 'html.parser'
    # проверка корректности результата BeautifulSoup
    if ticket_bs:
    # получение признаков из карточки текущего обьявления
        # bodyType      
        try: features['bodyType'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN
        # brand        
        try: features['brand'] = ticket_url.split('/')[-4].upper()
        except Exception: features['brand'] = np.NaN
        # car_url
        features['car_url'] = ticket_url
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN 
        # description
        try:
            rows = ticket_bs.find('div',class_='CardDescriptionHTML').find_all('span')
            features['description'] = '\n'.join([row.text for row in rows])
        except Exception: features['description'] = np.NaN
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
        # enginePower
        try: 
            enginePower = ticket_bs.find(
                'li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN            
        # equipment_dict 
        try: 
            complectation_groups = ticket_bs.find(
                'div',class_='ComplectationGroupsDesktop__row').find_all(
                'div',class_='ComplectationGroupsDesktop__group')
            features['equipment_dict'] = {
                group.text.split('•')[0]: group.text.split('•')[1:] for group in complectation_groups}
        except Exception: features['equipment_dict'] = np.NaN             
        # fuelType
        try: features['fuelType'] = ticket_bs.find(
            'li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        except Exception: features['fuel_type'] = np.NaN
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN            
        # model_name
        try: features['model_name'] = ticket_url.split('/')[-3].upper()
        except Exception: features['model_name'] = np.NaN            
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN
        # productionDate
        try: features['productionDate'] = ticket_bs.find(
            'li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'  
        # Владение
        try: features['Владение'] = ticket_bs.find('li',class_='CardInfoRow_owningTime').find_all('span')[1].text
        except Exception: features['Владение'] = np.NaN             
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN            
        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN               
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN            
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN         
        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN
        # modelDate              
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[2].text.strip() + ' ' +
                                     modelDate_tag[3].text.strip() + ' ' +
                                     modelDate_tag[4].text.strip())  
        except Exception: features['modelDate'] = np.NaN   
    return features

## Установка параметров, определение констант

In [4]:
# service=Service('C:/Users/GANSOR-PC/chromium/chromedriver.exe') # C:\Users\GANSOR-PC
executable_path = 'C:/Users/GANSOR-PC/chromium/chromedriver.exe'
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
data_columns = ['bodyType', 'brand', 'car_url', 'color', 'description', 'engineDisplacement', 'enginePower', 
                      'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                      'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                      'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'offerprice']

# Парсинг
## Парсинг вспомогательных данных
### Создаем и наполняем ***`marks_models `*** , получаем ***`marks_models_for_parsing`*** и 
***marks_models*** - словарь в которм   
ключ:  обозначения марки на сайте auto.ru  
значения: списки всех моделей для каждой марки.

***marks_models_for_parsing*** - содержит только марки, присутсвующие в валидационном наборе данных **test**

затем сохраняем / читаем сохраненный ***marks_models_for_parsing***

In [5]:
# marks_models = get_marks_models()
# marks_models_for_parsing = dict([(k, marks_models.get(k)) for k in marks_for_parsing])

# with open('./Project_7_data/marks_models_for_parsing.json', 'w') as f: json.dump(marks_models_for_parsing, f)
with open('./Project_7_data/marks_models_for_parsing.json') as f: marks_models_for_parsing = json.loads(f.read())

### Создаем и заполняем `model_generation_year`
***model_generation_year*** - **pd.DataFrame**  в котором:  
        *full_name* - полное название марки и модели с указанием поколения;   
        *bodytype* - тип кузова поколения модели;  
        *generation_year* - год начала выпуска поколения  

Этот DataFrame нужен для преобразования полного названия модели в год начала выпуска модели

In [6]:
%%time
# model_generation_year_all =  get_model_generation_year(marks_models_for_parsing)

# model_generation_year_all.to_csv('./Project_5_data/model_generation_year.csv',index=False)
model_generation_year = pd.read_csv('./Project_7_data/model_generation_year.csv')

Wall time: 31.7 ms


# Парсинг набора данных для обучения
## Получение данных по маркам, присутсвующим в валидационном наборе данных

In [7]:
train = pd.DataFrame(columns=data_columns)

In [8]:
marks_models_for_parsing.keys()

dict_keys(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI', 'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'])

In [11]:
%%time
if 1:
    option = webdriver.ChromeOptions()
    chrome_prefs = {}
    option.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
# def simple_parsing(data_columns,marks_models_for_parsing):
    for mark in {k: marks_models_for_parsing[k] for k in ['VOLKSWAGEN'] }:
#     for mark in marks_models_for_parsing: # 
        print(mark)
        for model in marks_models_for_parsing[mark][:3]:      
            model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/' + '?output_type=table'
        # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
            driver = webdriver.Chrome(executable_path,options=option)
            driver.maximize_window()        
            for pages_num in range(1,99): 
                if pages_num==1: page_url = model_url
                else:            page_url = model_url + '&page=' +  str(pages_num)  
                # получем html страницы
                driver.get(page_url)
                page_html = driver.execute_script("return document.body.innerHTML;")
                # создаем обьект bs4.BeautifulSoup из html страницы
                page_bs = BeautifulSoup(page_html, 'lxml') # 'html.parser'
                # список html-ек карточек на странице                                               
                tickets_on_page = page_bs.find_all('a', class_='ListingItemTitle__link') 
                # выход по исчерпанию страниц текущей модели текущей марки
                if not tickets_on_page: 
                    driver.quit()
                    break            
                # обработка карточек на странице
                for ticket in tickets_on_page:
                    # получаем url карточки текущего обьявления
                    ticket_url = ticket.get('href')
                    # извлекаем признаки и заполняем строку train-а
                    train.loc[len(train)] =get_features_from_ticket(ticket_url,driver)
            driver.quit()
            print ('|',len(train),end='') # <==============================             
        print ('|') # <==============================    
#     return train

# train = simple_parsing(data_columns,marks_models_for_parsing)

VOLKSWAGEN
| 78| 87| 102|
Wall time: 2min 59s


In [115]:
# train.to_csv('./Project_7_data/train_mod.csv',index=False)
# train = pd.read_csv('./Project_7_data/train_mod.csv')

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 23 columns):
bodyType               26 non-null object
brand                  27 non-null object
car_url                27 non-null object
color                  26 non-null object
description            26 non-null object
engineDisplacement     26 non-null object
enginePower            26 non-null object
equipment_dict         23 non-null object
fuelType               26 non-null object
mileage                26 non-null object
modelDate              26 non-null object
model_name             27 non-null object
numberOfDoors          0 non-null object
productionDate         26 non-null object
sell_id                27 non-null object
vehicleTransmission    26 non-null object
vendor                 27 non-null object
Владельцы              26 non-null object
Владение               8 non-null object
ПТС                    26 non-null object
Привод                 26 non-null object
Руль             

In [15]:
train

,bodyType,brand,car_url,color,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,...,productionDate,sell_id,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,offerprice
0,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,белый,"Автомобиль в отличном состоянии, вложения не т...",2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,195000,...,2013,1114677051,механическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,1650000
1,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серебристый,АВТОРАССРОЧКА НА АВТОМОБИЛИ С ПРОБЕГОМ 0%!\n\n...,3.0,204,{'Безопасность': ['Подушка безопасности водите...,Дизель,150920,...,2016,1115036990,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,2999000
2,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,коричневый,ЛОТ: 01315293\nЛокация №19\nФаворит Хофф\n\n\n...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,225457,...,2013,1115046248,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,1840000
3,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,коричневый,ЛОТ: 01299790\nЛокация №19\nФаворит Хофф\n\n\n...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,169867,...,2013,1105973842,механическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,1869000
4,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,чёрный,ЛОТ: 01316216\nЛокация №8\nFAVORIT MOTORS Peug...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,92617,...,2015,1114844447,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,2900000
5,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серый,Продажа от официального дилера ГК «АВТОМИР».\n...,2.0,180,{'Безопасность': ['Подушки безопасности боковы...,Дизель,191000,...,2016,1115042580,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,3420000
6,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1114836195,NaN,EUROPEAN,NaN,NaN,NaN,NaN,NaN,NaN
7,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,коричневый,АВТОРАССРОЧКА НА АВТОМОБИЛИ С ПРОБЕГОМ 0%!*\n\...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,298076,...,2012,1114836194,механическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,1663000
8,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,белый,Volkswagen Amarok I Пикап двойная кабина full-...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,138565,...,2013,1106041615,механическая,EUROPEAN,2 владельца,8 лет,Оригинал,полный,Левый,1639500
9,пикап двойная кабина,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серебристый,Volkswagen Amarok I Пикап двойная кабина full-...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,137234,...,2012,1102331043,автоматическая,EUROPEAN,1 владелец,9 лет,Оригинал,полный,Левый,1629500


# ========== TEST ================

In [16]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="b7e384bcddd7d5f66ac1d55a459a53c0")>

# ============= EXERCISE ==============

In [45]:
to_json = {'abc': 1, 
           'def': [2,3]}
to_json

{'abc': 1, 'def': [2, 3]}

In [52]:
with open('example.json', 'w') as f: json.dump(to_json, f)
with open('example.json') as f: from_json = json.loads(f.read())
    

{'abc': 1, 'def': [2, 3]}

In [ ]:
'''
train = pd.DataFrame(columns=data_columns)

for mark in marks_models_for_parsing.index:
    for model in marks_models_for_parsing[mark]:
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/'

    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,100): 
        
            if pages_num==1: params = {}
            else:            params = {'page': pages_num}
    
            res = requests.get(model_url + '?output_type=table', params=params, headers=headers)
            res.encoding = 'utf-8'
            time.sleep(0.5)
    
            # создаем обьект bs4.BeautifulSoup из html очередной страницы с обьявлениями по текущей модели текущей марки
            soup = BeautifulSoup(res.text, 'html.parser')  
    
            # список html-ек карточек на странице                                               
            tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') 
    
            if not tickets_on_page_list: # выход по исчерпанию страниц текущей модели текущей марки
                break
        
            # обработка карточек на странице
            for ticket in tickets_on_page_list:
                # получаем url карточки текущего обьявления
                ticket_url = ticket.get('href')
    
                # извлекаем признаки и заполняем строку train-а
                train.loc[len(train)] = get_features_from_ticket(ticket_url)
                ''';

%%time
for mark in {k: marks_models_for_parsing[k] for k in ['HONDA'] }: # 'VOLKSWAGEN' 'MITSUBISHI'
    print(mark)
    for model in marks_models_for_parsing[mark]:      
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/' + '?output_type=table'
    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,10): 
            if pages_num==1: page_url = model_url
            else:            page_url = model_url + '&page=' +  str(pages_num)  
            # получем html страницы
            driver = webdriver.Chrome(executable_path)
            driver.maximize_window()
            driver.get(page_url)
            page_html = driver.execute_script("return document.body.innerHTML;")
            # создаем обьект bs4.BeautifulSoup из html страницы
            page_bs = BeautifulSoup(page_html, 'html.parser') 
            # список html-ек карточек на странице                                               
            tickets_on_page = page_bs.find_all('a', class_='ListingItemTitle__link') 
            # выход по исчерпанию страниц текущей модели текущей марки
            if not tickets_on_page: 
                driver.quit()
                break            
            # обработка карточек на странице
            for ticket in tickets_on_page:
                # получаем url карточки текущего обьявления
                ticket_url = ticket.get('href')
                # извлекаем признаки и заполняем строку train-а
                train.loc[len(train)] =get_features_from_ticket(ticket_url,driver)
                time.sleep(0.5) 
            time.sleep(0.5) 
            driver.quit()
        print ('|',len(train),end='') # <==============================             
    print ('|') # <==============================       